In [ ]:
# Step0 Setup
import pandas as pd
import numpy as np
import json
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, brier_score_loss, roc_curve
from sklearn.linear_model import LogisticRegression

# create artifact directories if not existing
root = Path.cwd()
(art_dir := root / "artifacts").mkdir(exist_ok=True)
(art_data := art_dir / "data").mkdir(exist_ok=True)
(art_forecasts := art_dir / "forecasts").mkdir(exist_ok=True)
(art_metrics := art_dir / "metrics").mkdir(exist_ok=True)
(art_reports := art_dir / "reports").mkdir(exist_ok=True)

# load actual returns and directions from feature data
features_file = art_data / "features_monthly.csv"
if features_file.exists():
    df_features = pd.read_csv(features_file, parse_dates=[0])
    # Identify return and direction columns
    if "y_return" in df_features.columns:
        df_features["return"] = df_features["y_return"] / 100.0  # convert percent to fraction
        df_features["direction"] = df_features["y_direction"]
    elif "y_return_next_pct" in df_features.columns:
        df_features["return"] = df_features["y_return_next_pct"] / 100.0
        df_features["direction"] = df_features["y_direction_next"]
    else:
        raise KeyError("Return column not found in features data.")
else:
    raise FileNotFoundError(f"Features file not found: {features_file}")


In [ ]:
# Step1: Load forecast CSVs, validate and merge
models = ["linear", "arimax", "rf", "lstm", "baseline"]
feature_sets = ["F1", "F3"]
df_forecasts = {}

for fs in feature_sets:
    merged = None
    used_models = []
    for model in models:
        file = art_forecasts / f"{model}_{fs}.csv"
        if file.exists():
            df = pd.read_csv(file)
            # identify date column (first column)
            date_col = df.columns[0]
            df = df.rename(columns={date_col: "date"})
            df["date"] = pd.to_datetime(df["date"])
            # rename prediction columns
            if "y_pred_proba_up" in df.columns:
                df = df.rename(columns={"y_pred_proba_up": f"{model}_proba"})
            if "y_pred_dir" in df.columns:
                df = df.rename(columns={"y_pred_dir": f"{model}_dir"})
            if f"{model}_dir" in df.columns:
                df[f"{model}_dir"] = df[f"{model}_dir"].astype(int)
            # keep only date, proba, dir
            cols_to_keep = ["date"] + [col for col in df.columns if col.endswith("_proba") or col.endswith("_dir")]
            df = df[cols_to_keep]
            if merged is None:
                merged = df.copy()
            else:
                merged = pd.merge(merged, df, on="date", how="outer")
            used_models.append(model)
    if merged is None:
        raise FileNotFoundError(f"No forecast files found for feature set {fs}")
    merged = merged.sort_values("date").reset_index(drop=True)
    df_forecasts[fs] = {"df": merged, "models": used_models}


In [ ]:
# Step2: Compute ensemble forecasts
ensemble_results = {}
for fs in feature_sets:
    data = df_forecasts[fs]["df"]
    models_used = df_forecasts[fs]["models"]
    # Probability Averaging
    prob_cols = [f"{m}_proba" for m in models_used if f"{m}_proba" in data.columns]
    data["ensemble_mean_proba"] = data[prob_cols].mean(axis=1, skipna=True)
    data["ensemble_mean_dir"] = (data["ensemble_mean_proba"] >= 0.5).astype(int)
    ensemble_results[(fs, "mean")] = {
        "proba": data["ensemble_mean_proba"],
        "dir": data["ensemble_mean_dir"],
        "models_used": models_used
    }
    # Weighted Averaging by F1
    weights_f1 = {}
    total_f1 = 0.0
    for model in models_used:
        metrics_file = art_metrics / f"{model}_{fs}.json"
        if metrics_file.exists():
            m = json.load(open(metrics_file))
            oof_f1 = m.get("oof_f1", 0.0)
            weights_f1[model] = oof_f1
            total_f1 += oof_f1
    weights_f1 = {m: (w/total_f1 if total_f1>0 else 0) for m, w in weights_f1.items()}
    data["ensemble_w_f1_proba"] = 0.0
    for model, w in weights_f1.items():
        col = f"{model}_proba"
        if col in data.columns:
            data["ensemble_w_f1_proba"] += data[col] * w
    data["ensemble_w_f1_dir"] = (data["ensemble_w_f1_proba"] >= 0.5).astype(int)
    ensemble_results[(fs, "w_f1")] = {
        "proba": data["ensemble_w_f1_proba"],
        "dir": data["ensemble_w_f1_dir"],
        "models_used": list(weights_f1.keys()),
        "weights": weights_f1,
        "oof_weight_metric": "f1"
    }
    # Weighted Averaging by AUC
    weights_auc = {}
    total_auc = 0.0
    for model in models_used:
        metrics_file = art_metrics / f"{model}_{fs}.json"
        if metrics_file.exists():
            m = json.load(open(metrics_file))
            oof_auc = m.get("oof_auc", 0.0)
            weights_auc[model] = oof_auc
            total_auc += oof_auc
    weights_auc = {m: (w/total_auc if total_auc>0 else 0) for m, w in weights_auc.items()}
    data["ensemble_w_auc_proba"] = 0.0
    for model, w in weights_auc.items():
        col = f"{model}_proba"
        if col in data.columns:
            data["ensemble_w_auc_proba"] += data[col] * w
    data["ensemble_w_auc_dir"] = (data["ensemble_w_auc_proba"] >= 0.5).astype(int)
    ensemble_results[(fs, "w_auc")] = {
        "proba": data["ensemble_w_auc_proba"],
        "dir": data["ensemble_w_auc_dir"],
        "models_used": list(weights_auc.keys()),
        "weights": weights_auc,
        "oof_weight_metric": "auc"
    }
    # Majority Vote
    dir_cols = [f"{m}_dir" for m in models_used if f"{m}_dir" in data.columns]
    def majority_vote(row):
        votes = row[dir_cols].dropna().astype(int)
        if votes.size == 0:
            return 0
        ones = votes.sum()
        return 1 if ones > votes.size / 2 else 0
    data["ensemble_vote_dir"] = data.apply(majority_vote, axis=1)
    data["ensemble_vote_proba"] = data[dir_cols].mean(axis=1, skipna=True)
    ensemble_results[(fs, "vote")] = {
        "proba": data["ensemble_vote_proba"],
        "dir": data["ensemble_vote_dir"],
        "models_used": models_used
    }
# Save ensemble forecast CSVs
for fs in feature_sets:
    data = df_forecasts[fs]["df"]
    method_keys = {"mean": "ensemble_mean", "w_f1": "ensemble_w_f1", "w_auc": "ensemble_w_auc", "vote": "ensemble_vote"}
    for method_key, prefix in method_keys.items():
        name = f"ensemble_{method_key}_{fs}.csv"
        df_out = pd.DataFrame({
            "date": data["date"],
            "y_pred_proba_up": data[f"{prefix}_proba"],
            "y_pred_dir": data[f"{prefix}_dir"]
        })
        df_out.to_csv(art_forecasts / name, index=False)


In [ ]:
# Step3: Compute metrics for each ensemble, save JSON
from scipy.stats import chi2

# prepare actuals mapping by date
df_feat = df_features.copy()
date_col = df_feat.columns[0]
df_feat = df_feat.rename(columns={date_col: "date"})
df_feat["date"] = pd.to_datetime(df_feat["date"])
df_feat = df_feat.set_index("date")
y_true_series = df_feat["direction"]

for (fs, method_key), ens in ensemble_results.items():
    df_ens = df_forecasts[fs]["df"]
    dates = pd.to_datetime(df_ens["date"])
    y_true = y_true_series.reindex(dates).values
    y_pred = ens["dir"].values
    y_proba = ens["proba"].values
    mask = ~np.isnan(y_pred) & ~np.isnan(y_true)
    y_true = y_true[mask].astype(int)
    y_pred = y_pred[mask].astype(int)
    y_proba = y_proba[mask].astype(float)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    try:
        auc = roc_auc_score(y_true, y_proba)
    except ValueError:
        auc = float("nan")
    brier = brier_score_loss(y_true, y_proba)
    returns = df_feat["return"].reindex(dates)[mask].fillna(0).values
    strat_returns = returns * y_pred
    if strat_returns.std() > 0:
        sharpe = strat_returns.mean() / strat_returns.std() * np.sqrt(12)
    else:
        sharpe = float("nan")
    neg_returns = strat_returns[strat_returns < 0]
    if neg_returns.size > 0 and np.std(neg_returns, ddof=0) > 0:
        sortino = strat_returns.mean() / np.std(neg_returns, ddof=0) * np.sqrt(12)
    else:
        sortino = float("nan")
    if strat_returns.size > 0:
        try:
            top5 = np.percentile(strat_returns, 95)
            bot5 = np.percentile(strat_returns, 5)
            tail_top = strat_returns[strat_returns >= top5]
            tail_bot = strat_returns[strat_returns <= bot5]
            if tail_bot.size > 0:
                rachev = tail_top.mean() / abs(tail_bot.mean()) if abs(tail_bot.mean()) > 0 else float("nan")
            else:
                rachev = float("nan")
        except Exception:
            rachev = float("nan")
    else:
        rachev = float("nan")
    if "baseline_dir" in df_ens.columns:
        y_base = df_ens["baseline_dir"].reindex(dates).values[mask].astype(int)
        n01 = int(np.sum((y_pred == y_true) & (y_base != y_true)))
        n10 = int(np.sum((y_pred != y_true) & (y_base == y_true)))
        if n01 + n10 > 0:
            mcnemar_stat = (abs(n01 - n10) - 0)**2 / (n01 + n10)
            mcnemar_p = 1 - chi2.cdf(mcnemar_stat, 1)
        else:
            mcnemar_stat = None
            mcnemar_p = None
    else:
        mcnemar_stat = None
        mcnemar_p = None
    dm_stat = None
    dm_p = None
    metrics_dict = {
        "featureset": fs,
        "ensemble_method": method_key,
        "models_used": ens.get("models_used", []),
        "n_models": len(ens.get("models_used", [])),
        "weighting_scheme": ("equal" if method_key=="mean" else ("majority_vote" if method_key=="vote" else "weighted")),
        "oof_weight_metric": ens.get("oof_weight_metric", None),
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "auc": auc,
        "brier": brier,
        "sharpe": sharpe,
        "sortino": sortino,
        "rachev": rachev,
        "DM_stat": dm_stat,
        "DM_pvalue": dm_p,
        "McNemar_stat": mcnemar_stat,
        "McNemar_pvalue": mcnemar_p
    }
    json_name = art_metrics / f"ensemble_{method_key}_{fs}.json"
    with open(json_name, "w") as f:
        json.dump(metrics_dict, f, indent=2)


In [ ]:
# Step4: Plots for each ensemble
for fs in feature_sets:
    data = df_forecasts[fs]["df"]
    dates = pd.to_datetime(data["date"])
    # find best model by F1
    best_model = None
    best_f1 = -1
    for model in df_forecasts[fs]["models"]:
        metrics_file = art_metrics / f"{model}_{fs}.json"
        if metrics_file.exists():
            m = json.load(open(metrics_file))
            if m.get("f1",0) > best_f1:
                best_f1 = m["f1"]
                best_model = model
    if best_model is None:
        continue
    actual_returns = df_feat["return"].reindex(dates).fillna(0).values
    cum_bh = np.cumprod(1 + actual_returns) - 1
    best_dir = data[f"{best_model}_dir"].values
    cum_best = np.cumprod(1 + actual_returns * best_dir) - 1
    for method_key in ["mean", "w_f1", "w_auc", "vote"]:
        ens = ensemble_results[(fs, method_key)]
        # ROC Curve
        y_true = df_feat["direction"].reindex(dates).fillna(0).astype(int).values
        y_proba = ens["proba"].values
        mask = ~np.isnan(y_proba)
        y_true_m = y_true[mask]
        y_proba_m = y_proba[mask]
        if len(np.unique(y_true_m)) > 1:
            fpr, tpr, _ = roc_curve(y_true_m, y_proba_m)
            plt.figure()
            plt.plot(fpr, tpr, label=f"{method_key} (AUC={roc_auc_score(y_true_m,y_proba_m):.2f})")
            plt.plot([0,1], [0,1], 'k--', lw=0.5)
            plt.title(f"ROC Curve - Ensemble {method_key} {fs}")
            plt.xlabel("False Positive Rate")
            plt.ylabel("True Positive Rate")
            plt.legend()
            plt.savefig(art_reports / f"90_ROC_{method_key}_{fs}.png")
            plt.close()
        # Cumulative Returns
        ens_dir = ens["dir"].values
        strat_returns = actual_returns * ens_dir
        cum_ens = np.cumprod(1 + strat_returns) - 1
        plt.figure(figsize=(6,4))
        plt.plot(dates, cum_bh, label="Buy & Hold")
        plt.plot(dates, cum_best, label=f"Best Model: {best_model}")
        plt.plot(dates, cum_ens, label=f"Ensemble {method_key}")
        plt.title(f"Cumulative Returns - Ensemble {method_key} {fs}")
        plt.ylabel("Cumulative Return")
        plt.legend()
        plt.tight_layout()
        plt.savefig(art_reports / f"90_CumRet_{method_key}_{fs}.png")
        plt.close()
        # Bar metrics (Accuracy, Precision, Recall, F1) vs best model and baseline
        metrics_json = {}
        ens_metrics_file = art_metrics / f"ensemble_{method_key}_{fs}.json"
        if ens_metrics_file.exists():
            metrics_json["ensemble"] = json.load(open(ens_metrics_file))
        best_metrics_file = art_metrics / f"{best_model}_{fs}.json"
        if best_metrics_file.exists():
            metrics_json["best_model"] = json.load(open(best_metrics_file))
        base_metrics_file = art_metrics / f"baseline_{fs}.json"
        if base_metrics_file.exists():
            metrics_json["baseline"] = json.load(open(base_metrics_file))
        if "ensemble" in metrics_json and "best_model" in metrics_json and "baseline" in metrics_json:
            labels = ["Accuracy", "Precision", "Recall", "F1"]
            ens_vals = [metrics_json["ensemble"].get(l.lower(), 0) for l in labels]
            best_vals = [metrics_json["best_model"].get(l.lower(), 0) for l in labels]
            base_vals = [metrics_json["baseline"].get(l.lower(), 0) for l in labels]
            x = np.arange(len(labels))
            width = 0.25
            plt.figure(figsize=(6,4))
            plt.bar(x - width, ens_vals, width, label="Ensemble")
            plt.bar(x, best_vals, width, label="Best Model")
            plt.bar(x + width, base_vals, width, label="Baseline")
            plt.xticks(x, labels)
            plt.ylabel("Score")
            plt.title(f"Metrics Comparison - Ensemble {method_key} {fs}")
            plt.legend()
            plt.tight_layout()
            plt.savefig(art_reports / f"90_MetricsBar_{method_key}_{fs}.png")
            plt.close()


In [ ]:
# Step5: Overview table for all ensembles
overview_rows = []
metrics_files = art_metrics.glob("ensemble_*.json")
for jf in metrics_files:
    m = json.load(open(jf))
    row = {
        "featureset": m.get("featureset"),
        "method": m.get("ensemble_method"),
        "accuracy": m.get("accuracy"),
        "precision": m.get("precision"),
        "recall": m.get("recall"),
        "f1": m.get("f1"),
        "auc": m.get("auc"),
        "brier": m.get("brier"),
        "sharpe": m.get("sharpe"),
        "sortino": m.get("sortino"),
        "rachev": m.get("rachev")
    }
    overview_rows.append(row)
if overview_rows:
    df_overview = pd.DataFrame(overview_rows)
    df_overview = df_overview.sort_values(["featureset", "method"]).reset_index(drop=True)
    csv_file = art_reports / "90_uebersicht_ensembles.csv"
    df_overview.to_csv(csv_file, index=False)
    fig, ax = plt.subplots(figsize=(8,2))
    ax.axis('off')
    table_data = df_overview.round(3)
    mpl_table = ax.table(cellText=table_data.values, colLabels=table_data.columns, loc='center')
    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(8)
    mpl_table.scale(1.2, 1.2)
    plt.tight_layout()
    plt.savefig(art_reports / "90_uebersicht_ensembles.png")
    plt.close()
